## Set up API and some functions

In [2]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [ ]:
import openai

# see: https://platform.openai.com/docs/quickstart
openai.api_key = ""  # add your key here
from IPython.display import display, Markdown
import requests
import json

In [4]:
# Define the DotDict class
class DotDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __init__(self, dct):
        for key, value in dct.items():
            if hasattr(value, "keys"):
                value = DotDict(value)
            self[key] = value

In [5]:
def clinical_trials_api(condition, terms, intervention, topK=5):
    assert topK <= 10

    # Define the payload for the request
    payload = {
        "query.cond": condition,
        "query.term": terms,
        "query.intr": intervention,
        # "filter.overallStatus": 'COMPLETED',
    }

    # Define the URL and make the request
    url = "https://clinicaltrials.gov/api/v2/studies"
    response = requests.get(url, params=payload)

    trial_ids = []
    clinical_trial_data = []

    # Check if the request was successful
    if response.status_code == 200:
        try:
            data = response.json()
            studies = data.get("studies", [])[:topK]

            for idx, study_data in enumerate(studies):
                study = DotDict(study_data)
                try:
                    # Get the study number (nctId)
                    nct_id = study.protocolSection.identificationModule.nctId
                    trial_ids.append("https://clinicaltrials.gov/study/" + str(nct_id))
                    clinical_trial_data.append(
                        "Clinical Trial Details #{} in Nested JSON format:".format(
                            idx + 1
                        )
                    )
                    clinical_trial_data.append(json.dumps(study))
                except (KeyError, AttributeError):
                    continue

            # Save clinical trial data as a string in results_dict
            clinical_trial_data = "\n".join(clinical_trial_data)

        except requests.exceptions.JSONDecodeError:
            print("Failed to decode JSON. Here is the raw response:")
            print(response.text)
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)

    # Print the collected clinical trial data
    print("The following top {} clinical trials are pulled:".format(topK))
    print("\n".join(trial_ids))
    # print(clinical_trial_data)

    return trial_ids, clinical_trial_data

In [6]:
def chatgpt_summary(context):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # "o1-mini" (more expensive) or "gpt-4o-mini" (cheaper)
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "Please summarize the most clinically relevant information out of these nested JSON responses from a Clinical Trial Database for various related clinical trials. Please start with summary right away without any small talk. This is the trial data: {}".format(
                    context
                ),
            },
        ],
    )
    display(Markdown(response["choices"][0]["message"]["content"]))
    return response

In [7]:
def chatgpt_ask(context, question):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # "o1-mini" (more expensive) or "gpt-4o-mini" (cheaper)
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "Please thoroughly understand clinically important information encoded in the following nested JSON responses from a Clinical Trial Database for various related clinical trials. Please answer the subsequent questions based on this data. Please be factual and always cite the clinical trial source when you use a certain piece of information from there. Here is the trial data: {}".format(
                    context
                ),
            },
            {
                "role": "user",
                "content": "Now answer the following question: {}".format(question),
            },
        ],
    )
    display(Markdown(response["choices"][0]["message"]["content"]))
    return response

# Demo

In [8]:
condition = "infarction"
terms = "diabetes"
intervention = "heart attack"
top_number_of_trials_to_consider = 10 # up to 10 supported for now

In [9]:
clinical_trials_context = clinical_trials_api(
    condition, terms, intervention, topK=top_number_of_trials_to_consider
)

The following top 10 clinical trials are pulled:
https://clinicaltrials.gov/study/NCT03620266
https://clinicaltrials.gov/study/NCT06017544
https://clinicaltrials.gov/study/NCT03646357
https://clinicaltrials.gov/study/NCT04188457
https://clinicaltrials.gov/study/NCT03658031
https://clinicaltrials.gov/study/NCT01887080
https://clinicaltrials.gov/study/NCT06245980
https://clinicaltrials.gov/study/NCT00725127
https://clinicaltrials.gov/study/NCT00921960
https://clinicaltrials.gov/study/NCT05535582


In [10]:
response = chatgpt_summary(clinical_trials_context)

### Summary of Clinical Trials

1. **BioDiaMI (NCT03620266)**  
   - **Objective**: Investigate the effects of bilberry and oat intake on lipid profiles, inflammation, and exercise capacity in patients with Type 2 Diabetes Mellitus (T2DM) or Myocardial Infarction (MI).  
   - **Design**: Randomized, double-blind, placebo-controlled trial with an estimated enrollment of 900 participants.  
   - **Primary Endpoint**: LDL cholesterol change after three months.  
   - **Eligibility**: Adults with T2DM and/or within three years of MI.  
   - **Status**: Recruiting.

2. **GLP-1RAs and SGLT2is Therapy (NCT06017544)**  
   - **Objective**: Evaluate the combination therapy of GLP-1 receptor agonists (GLP-1RAs) and SGLT2 inhibitors (SGLT2is) on major adverse cardiovascular events (MACE) in T2DM patients post-MI.  
   - **Design**: Observational study with an estimated enrollment of 257 participants.  
   - **Primary Outcome**: Composite of MACE, including mortality and heart failure.  
   - **Eligibility**: Adults with T2DM hospitalized due to AMI.  
   - **Status**: Recruiting.

3. **BETAMI (NCT03646357)**  
   - **Objective**: Study the efficacy of beta-blocker therapy post-AMI in patients without reduced left ventricular systolic function.  
   - **Design**: Randomized, open-label trial. Estimated enrollment of 2895 participants.  
   - **Primary Endpoint**: Composite of death, recurrent MI, heart failure, etc. over 6 months to 6 years.  
   - **Eligibility**: Adults ≥18 years diagnosed with AMI treated with PCI or thrombolysis.  
   - **Status**: Active, not recruiting.

4. **DiVa (NCT04188457)**  
   - **Objective**: Evaluate intensive outpatient monitoring versus usual care for neuro-cardiovascular diseases post-MI.  
   - **Design**: Observational study with an estimated enrollment of 1720 participants.  
   - **Primary Outcome**: Cost-utility ratio associated with intensive monitoring.  
   - **Eligibility**: Adults with recent cardiac events.  
   - **Status**: Recruiting.

5. **Dapagliflozin Study (NCT03658031)**  
   - **Objective**: Assess the impact of dapagliflozin on the progression from prediabetes to T2DM in post-MI patients.  
   - **Design**: Randomized, double-blind trial with an estimated enrollment of 576 participants.  
   - **Primary Outcome**: Incidence of T2DM at 36 months.  
   - **Eligibility**: Adults with prediabetes and recent MI.  
   - **Status**: Not yet recruiting.

6. **Microcurrent in Cardiac Rehab (NCT01887080)**  
   - **Objective**: Investigate the effects of microcurrent therapy in a home-based cardiac rehabilitation program post-MI.  
   - **Design**: Completed interventional trial with a sample size of 40.  
   - **Primary Outcome**: Change in cardiorespiratory fitness at 8 weeks.  
   - **Eligibility**: Adults aged 40-75 with stabilized heart disease post-MI.  
   - **Status**: Completed.

7. **Role of SGLT2I (NCT06245980)**  
   - **Objective**: Investigate the role of SGLT2 inhibitors in diabetic patients post-MI.  
   - **Design**: Observational cohort study with an estimated enrollment of 1200 participants.  
   - **Primary Outcomes**: MACE, recurrent MI, heart failure readmission over 24 months.  
   - **Eligibility**: Diabetic patients with acute MI.  
   - **Status**: Recruiting.

8. **Chronotherapy with Aspirin (NCT00725127)**  
   - **Objective**: Examine the effects of taking low-dose aspirin in the morning versus at bedtime on preventing cardiovascular events.  
   - **Design**: Randomized trial with an estimated 3200 participants.  
   - **Primary Outcome**: Effects on cardiovascular events over 5 years.  
   - **Eligibility**: Adults ≥50 years with impaired fasting glucose or T2DM.  
   - **Status**: Recruiting.

9. **STOP-HF Study (NCT00921960)**  
   - **Objective**: Identify and prevent left ventricular dysfunction in at-risk asymptomatic individuals using natriuretic peptides.  
   - **Design**: Completed interventional study with an enrollment of 1378 participants.  
   - **Primary Outcome**: Prevalence and severity of left ventricular dysfunction after 5 years.  
   - **Eligibility**: Adults with cardiovascular risk factors.  
   - **Status**: Completed.

10. **Beyond-SMuRFs Study (NCT05535582)**  
    - **Objective**: Investigate clinical, laboratory, and imaging biomarkers associated with acute myocardial infarction in patients without standard modifiable risk factors.  
    - **Design**: Observational cohort study with a planned 750 participants.  
    - **Primary Outcome**: Differences in clinical features and biomarkers between patients with and without SMuRFs over 2 years.  
    - **Eligibility**: Adults >25 years presenting with AMI.  
    - **Status**: Active, not recruiting.

### Note
This summary captures the key clinical aspects of the listed trials, including objectives, design, primary outcomes, eligibility criteria, and current status.

In [11]:
question = """

What are the innovative ideas have been introduced in the clinical trials recently that are quite differentiating from the regular/conventional way
of executing clinical trials? Do the trials look hopeful at all? Are any of them still recruiting?

Also, answer the following questions please:
1) Which trial has more men registered than women?
2) Which trial deals with interventions that are generally thought to hold the most number of side affects than the others?
3) Can you also please return the SMILE strings of drugs used in each of the trials, if any.
4) If there is one thing you learnt from this list of trials that you haven't had an idea of before, what is that one learning?

"""
response = chatgpt_ask(clinical_trials_context, question.strip())

### Innovative Ideas in Recent Clinical Trials

Several clinical trials in the dataset employ innovative methodologies that differentiate them from conventional trial execution:

1. **Combination Therapies**: Trials like NCT06017544 (GLP-1RAs and SGLT2is Combination Therapy) explore combining medications with complementary mechanisms to improve outcomes in patients post-myocardial infarction, which is more tailored to individual patient needs.

2. **Dietary Interventions**: The NCT03620266 trial, which examines the effects of bilberry and oat intake on cardiovascular health, takes a non-pharmacological approach to intervention, focusing on dietary changes rather than solely drug interventions. This trial acknowledges the importance of nutrition in managing chronic conditions like type 2 diabetes and myocardial infarction.

3. **Use of Biomarkers**: The NCT05535582 study focuses on identifying patients without standard modifiable risk factors (SMuRFs) for acute myocardial infarction through clinical, laboratory, and echocardiographic biomarkers. This screening could lead to new predictive models for at-risk patients that deviate from traditional risk assessments based on conventional factors.

4. **Chronotherapy**: The NCT00725127 trial (CARING Study) investigates the timing of aspirin administration and its impact on cardiovascular events in patients with impaired fasting glucose or diabetes, which takes advantage of chronobiology.

**Hopefulness and Recruitment Status**:
Many of these trials appear hopeful as they address significant issues in cardiovascular care and diabetes management, potentially leading to improved health outcomes. As of the latest information:
- Clinical trials such as NCT03620266, NCT06017544, NCT04188457, and NCT06245980 are actively recruiting participants.

### Specific Questions Answered

1. **Trial with More Men Registered than Women**:
   - Unfortunately, the provided data does not include detailed information about the gender distribution of registered participants for each trial. Thus, I cannot specify which trial has more men than women based on the given information.

2. **Trial Dealing with Interventions with More Side Effects**:
   - NCT04188457 (BETAMI Study) appears to deal with beta-blockers, which are known to have a range of side effects, including fatigue, dizziness, and in some cases, more severe complications. This contrasts with dietary supplements or observational studies, which typically have a lower risk of side effects compared to pharmacological interventions.

3. **SMILE Strings of Drugs Used in Trials**:
   - **NCT03620266**: No specific drug mentioned; focuses on dietary supplements (bilberry, oat).
   - **NCT06017544**: GLP-1RAs, SGLT2 inhibitors.
   - **NCT04188457**: Beta-blockers.
   - **NCT03658031**: Dapagliflozin.
   - **NCT01887080**: Microcurrent (not a drug, but a therapy).
   - **NCT00725127**: Aspirin.
   - **NCT00921960**: Not applicable as it's a screening study.

4. **New Learning from the List of Trials**:
   - One notable learning is the increasing movement towards integrating lifestyle changes and complementary therapies in conjunction with or as alternatives to traditional pharmacological treatments in managing chronic diseases. The use of guidelines that encompass lifestyle interventions to reduce cardiovascular risk signifies a shift towards holistic care approaches. 

Overall, the trials reflect a blend of emerging pharmacological strategies, novel therapeutic approaches, and an emphasis on personalized medicine, providing exciting prospects for addressing complex health challenges.